Условия

---

$i = 4$

$i = 5$

$\lambda = \frac{1+i}{4} = \frac{5}{4} - \text{среднее кол-во сообщений в минуту}$

$n = 3+[\frac{i+j}{8}] = 4 - \text{кол-во каналов}$ 

$\tau = \frac{5}{5+j} = 0.5 - \text{среднее время обработки сообщения}$



In [11]:
from math import factorial

lbd = 5/4 
tau = 0.5

mu = 1/tau
rho = lbd/mu

ch_n = 5

print('\nТеоретические значения.')
print('Rho/n = ',rho/ch_n-1)
if rho/ch_n-1 < 1 :
    print('Предельные вероятности существуют')

    p_0 = 0 
    for i in range(0,ch_n):
        p_0 += rho**i/factorial(i)
    
    p_0 += rho**ch_n/(factorial(ch_n-1)*(ch_n-1-rho))
    p_0 = p_0 ** -1
    print('p_0: ',p_0)
    p_q = (rho**ch_n)/(factorial(ch_n-1)*(ch_n-1-rho))*p_0
    print('Вероятность попадания в очередь: ', p_q)
    print('Среднее число занятых каналов: ',rho)
    L_q = (rho**ch_n*p_0)/((ch_n-1)*factorial(ch_n-1)*(1-(rho/(ch_n-1)))**2)
    print('Среднее число заявок в очереди: ',L_q)
    print('Среднее число заявок в СМО: ',L_q+rho)
else:
    print('Очередь растет до бесконечности')



Теоретические значения.
Rho/n =  -0.875
Предельные вероятности существуют
p_0:  0.535177824807722
Вероятность попадания в очередь:  0.0006301053657151705
Среднее число занятых каналов:  0.625
Среднее число заявок в очереди:  0.0007467915445513131
Среднее число заявок в СМО:  0.6257467915445513


In [12]:
import numpy 
import cProfile
from tqdm import tqdm
delta_t = 0.001
t_meansurment_val = (delta_t**-1)
class Channel:
    def __init__(self):
        self.work_timer = 0
        self.state = True
    
    def is_empty(self):
        return self.state
    
    def set_work(self,work_time):
        if self.state:
            self.state = False
            self.work_timer = work_time
    
    def step(self):
        if not self.state and self.work_timer:
            self.work_timer -= delta_t
            if self.work_timer < delta_t:
                self.state = True
                self.work_timer = 0

class Network:
    def __init__(self,ch_count,intensivity,avr_compute_time):
        self.intensivity = intensivity
        self.avr_compute_time = avr_compute_time
        self.rnd_gen = numpy.random.default_rng()
        self.next_msg = self.rnd_gen.exponential(1/self.intensivity)
        self.total_time = 0
        self.total_task = 0
        self.miss_task = 0
        self.channels = [] 
        self.task_times = []
        self.empty_channels = []
        self.queue_size = []
        self.queue = []
        for i in range(0,ch_count):
            self.channels.append(Channel())

### Возвращает номер свободного канала, если все каналы заняты возвращает -1
    def check_free_channel(self):
        for index,i in enumerate(self.channels):
            if i.is_empty():
                return index 
        return -1

    def empty_channels_count(self):
        state_count = 0
        for i in self.channels:
            state_count += int(i.is_empty())
        return state_count

    def step(self):
        free_channel = self.check_free_channel()
        if not (free_channel == -1) and self.queue:
            self.channels[free_channel].set_work(self.queue.pop())

        if self.next_msg < delta_t:
            self.new_task()
        else:
            self.next_msg -= delta_t

        for i in self.channels:
            i.step()

        self.total_time += delta_t

        self.empty_channels.append(self.empty_channels_count())
        self.queue_size.append(len(self.queue))

    def new_task(self):
        self.next_msg = self.rnd_gen.exponential(1/self.intensivity)
        free_channel = self.check_free_channel()
        task_time = self.rnd_gen.exponential(self.avr_compute_time)
        if not (free_channel == -1):
            self.total_task +=1
            self.task_times.append(task_time)
            self.channels[free_channel].set_work(task_time)
        else:
            if len(self.queue) > len(self.channels)+1:
                self.miss_task += 1
            else:
                self.total_task +=1
                self.task_times.append(task_time)
                self.queue.append(task_time)

    def get_avr_task_time(self):
        return numpy.mean(self.task_times)
    
    def get_avr_empty_chanells(self):
        return numpy.mean(self.empty_channels)
    
    def get_total_tasks(self):
        return self.total_task

    def get_missed_tasks(self):
        return self.miss_task

    def get_mean_queue_size(self):
        return numpy.mean(self.queue_size)

n = t_meansurment_val*60*24
net = Network(4,5/4,0.5)
for i in range(0,int(n)):
    net.step()

print('\nПрактические значения.')
print('Обработано сообщений: ',net.get_total_tasks())
print('Отказано в обработке: ',net.get_missed_tasks())
print('Среднее время обработки: ',net.get_avr_task_time())
print('Средний размер очереди: ',net.get_mean_queue_size())
print('Среднее кол-во занятых каналов: ',4-net.get_avr_empty_chanells())
print('Абсолютная пропускная способность: ',t_meansurment_val*(net.get_total_tasks()-net.get_missed_tasks())/n)
print('Относительная пропускная способность: ',(net.get_total_tasks()-net.get_missed_tasks())/net.get_total_tasks())
print('Вероятность отказа: ',1-(net.get_total_tasks()-net.get_missed_tasks())/net.get_total_tasks())


Практические значения.
Обработано сообщений:  1862
Отказано в обработке:  0
Среднее время обработки:  0.49090183195116605
Средний размер очереди:  0.00022916666666666666
Среднее кол-во занятых каналов:  0.632829861111111
Абсолютная пропускная способность:  1.2930555555555556
Относительная пропускная способность:  1.0
Вероятность отказа:  0.0
